# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [13]:
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
df = pd.read_csv(url, index_col=0)
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [14]:
df.shape

(6000, 61)

In [15]:
X = df.drop(['url', 'Popular'], axis=1)
y = df['Popular']

In [16]:
y.mean()

0.5

In [17]:
#Normalize

from sklearn.preprocessing import Normalizer
X_norm=pd.DataFrame(Normalizer().fit_transform(X))

# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [22]:
#Estimate a Decision Tree Classifier and a Logistic Regresion
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

models = {'logistic regression': LogisticRegression(),
          'decision tree': DecisionTreeClassifier()}

for model in models.keys():
    models[model].fit(X_train, y_train)

    y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)
y_pred.head()

#Evaluate using F1 score and accuracy
from sklearn.metrics import mean_squared_error
from sklearn import metrics
for model in models.keys():
    print(model)
    print('f1 score:',metrics.f1_score(y_pred[model], y_test), 'Accuracy score:',metrics.accuracy_score(y_pred[model], y_test))

logistic regression
f1 score: 0.6104529616724739 Accuracy score: 0.6273333333333333
decision tree
f1 score: 0.5417218543046358 Accuracy score: 0.5386666666666666


# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [23]:
n_estimators=100
np.random.seed(123)
n_samples=X_train.shape[0]
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(n_estimators)]

In [25]:
#Create 100 Logistic regressions

np.random.seed(123)
seeds = np.random.randint(1, 10000, size=n_estimators)
log = {}
for i in range(n_estimators):
    log[i] = LogisticRegression(random_state=seeds[i])
    log[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

    
y_pred_df = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    y_pred_df.iloc[:, i] = log[i].predict(X_test)

In [26]:
#Create 100 Decision Trees where max_depth=None

np.random.seed(123)
seeds = np.random.randint(1, 10000, size=n_estimators)
trees = {}
for i in range(n_estimators):
    trees[i] = DecisionTreeClassifier(max_features="sqrt", max_depth=None, random_state=seeds[i])
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

    
y_pred_df1 = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    y_pred_df1.iloc[:, i] = trees[i].predict(X_test)


In [27]:
#Create 100 Decision Trees where max_depth=2

np.random.seed(123)
seeds = np.random.randint(1, 10000, size=n_estimators)
trees2 = {}
for i in range(n_estimators):
    trees2[i] = DecisionTreeClassifier(max_features="sqrt", max_depth=2, random_state=seeds[i])
    trees2[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

    
y_pred_df2 = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    y_pred_df2.iloc[:, i] = trees2[i].predict(X_test)


# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [31]:
f=pd.DataFrame(columns=['model','accuracy score',"f1 score"],data=[])
f.shape[0]
y_pred = pd.DataFrame(index=X_test.index, columns=list(range(300)))
for i in range(0,100):
    y_pred.iloc[:, i] = y_pred_df[i]
for i in range(100,200):
    y_pred.iloc[:, i] = y_pred_df1[i-100]
for i in range(200,300):
    y_pred.iloc[:, i] = y_pred_df1[i-200]
y_pred1 = (y_pred.sum(axis=1) >= (300 / 2)).astype(np.int)
accuracy=metrics.accuracy_score(y_pred1, y_test)
f1=metrics.f1_score(y_pred1, y_test)
f.loc[0] = ['majority voting',accuracy,f1]
f

,model,accuracy score,f1 score
0,majority voting,0.630667,0.621067


# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [35]:
t = range(0,300,5)
j=1
for i in t:
    y_pred2 = (y_pred.sum(axis=1) >= i).astype(np.int)
    accuracy=metrics.accuracy_score(y_pred2, y_test)
    f1=metrics.f1_score(y_pred2, y_test)
    f.loc[j] = [ "percentage models: %.2f%s"%((i/300)*100, '%'),accuracy,f1]
    j+=1
    
#select the threshold that maximizes the F1-Score
f[f['f1 score']==np.max(f['f1 score'])]

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


,model,accuracy score,f1 score
14,percentage models: 21.67%,0.61,0.709677


# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [37]:
samples_oob = []
for sample in samples:
    samples_oob.append(sorted(set(range(n_samples)) - set(sample)))

In [38]:
error = np.zeros(n_estimators)
for i in range(n_estimators):
    y_pred_df3 = trees[i].predict(X_train.iloc[samples_oob[i]])
    error[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_df3)

error1 = np.zeros(n_estimators)
for i in range(n_estimators):
    y_pred_df4 = trees2[i].predict(X_train.iloc[samples_oob[i]])
    error1[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_df4)

error2 = np.zeros(n_estimators)
for i in range(n_estimators):
    y_pred_df5 = log[i].predict(X_train.iloc[samples_oob[i]])
    error2[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_df5)


errors=np.concatenate((error,error1,error2))

In [40]:
f1=pd.DataFrame(columns=['model','accuracy score','f1 score'],data=[])
f1.shape[0]
alpha = (1 - errors) / (1 - errors).sum()
weighted_sum = ((y_pred) * alpha).sum(axis=1)
y_pred3 = (weighted_sum >= 0.5).astype(np.int)
accuracy=metrics.accuracy_score(y_pred3, y_test)
f1_score=metrics.f1_score(y_pred3, y_test)
f1.loc[0] = ['weighted voting',accuracy,f1_score]
f1

,model,accuracy score,f1 score
0,weighted voting,0.630667,0.621067


# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [41]:
t = range(0,100,5)
j=1
for i in t:
    y_pred4 = (weighted_sum >= i/100).astype(np.int)
    accuracy=metrics.accuracy_score(y_pred4, y_test)
    f1_score=metrics.f1_score(y_pred4, y_test)
    f1.loc[j] = [ "probability of the weighted voting: %.0f%s"%(i, '%'),accuracy,f1_score]
    j+=1

#select the threshold that maximizes the F1-Score
f1[f1['f1 score']==np.max(f1['f1 score'])]

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


,model,accuracy score,f1 score
6,probability of the weighted voting: 25%,0.620667,0.707455


# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score

In [44]:
#Estimate a logistic regression using as input the estimated classifiers

from sklearn.linear_model import LogisticRegressionCV

X_train_2 = pd.DataFrame(index=X_train.index, columns=list(range(300)))

for i in range(n_estimators):
    X_train_2[i] = trees[i].predict(X_train)
for i in range(100,200):
    X_train_2.iloc[:, i] = trees2[i-100].predict(X_train)
for i in range(200,300):
    X_train_2.iloc[:, i] = log[i-200].predict(X_train)

f2=pd.DataFrame(columns=['model','accuracy score',"f1 score"],data=[])
f2.shape[0]
lr = LogisticRegressionCV(cv=10)
lr.fit(X_train_2, y_train)
y_pred5 = lr.predict(y_pred)
accuracy=metrics.accuracy_score(y_pred5, y_test)
f1_score=metrics.f1_score(y_pred5, y_test)
f2.loc[0] = ['Logistic regression using as input the estimated classifiers',accuracy,f1_score]
f2

,model,accuracy score,f1 score
0,Logistic regression using as input the estimat...,0.622667,0.60749


In [45]:
#Modify the probability threshold such that maximizes the F1-Score

t = range(0,100,5)
j=1
for i in t:
    y_pred6 = np.where(lr.predict_proba(y_pred)[:,1] >= i/100, 1, 0)
    accuracy=metrics.accuracy_score(y_pred6, y_test)
    f1_score=metrics.f1_score(y_pred6, y_test)
    f2.loc[j] = [ "Logistic regression threshold: %.0f%s"%(i, '%'),accuracy,f1_score]
    j+=1
#select the one that maximizes the F1-Score
f2[f2['f1 score']==np.max(f2['f1 score'])]

,model,accuracy score,f1 score
1,Logistic regression threshold: 0%,0.504,0.670213


In [ ]:
#########